<a href="https://colab.research.google.com/github/christianwarmuth/transformer_adapter_bias_evaluation/blob/main/code/adapter_evaluation/adapter_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jul 27 11:10:43 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    30W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount("/content/gdrive/", force_remount=True)

Mounted at /content/gdrive/


In [ ]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
#!pip install transformers==3.0.2
!pip3 install -r /content/gdrive/MyDrive/master_hpi/NLP_Project/requirements.txt

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-cgymg8un
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-cgymg8un
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 128 kB 8.9 MB/s 
     |████████████████████████████████| 6.1 MB 22.8 MB/s 
     |████████████████████████████████| 156 kB 44.2 MB/s 
     |████████████████████████████████| 82 kB 1.2 MB/s 
     |████████████████████████████████| 547 kB 49.9 MB/s 
     |████████████████████████████████| 58 kB 6.6 MB/s 
     |████████████████████████████████| 294 kB 41.7 MB/s 
     |████████████████████████████████| 118 kB 47.2 MB/s 
     |████████████████████████████████| 509 kB 46.2 MB/s 
     |████████████████████████████████| 689 kB 34.3 MB/s 
     |████████████████████████████████| 69 kB 8.0 MB/s 
     |█████████████████████████

In [ ]:
import sys
from collections import defaultdict 
from collections import defaultdict 
from transformers import BertForMaskedLM, BertModel, BertTokenizer, BertForSequenceClassification

sys.path.append('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/')
path = "/content/gdrive/MyDrive/master_hpi/NLP_Project/code/"

In [ ]:
#adapter_list = ["sst"]
#for adapter in adapter_list:
#  print("!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1")

## Sentiment Bert


In [ ]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/')
except OSError as e:
        pass

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sentiment_bert/SentimentBert.pth --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sentiment_bert/summary.json

## SST Adapter


In [ ]:
import os
try:
    os.makedirs('/content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/')
except OSError as e:
        pass

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/evaluation.py --load-path_model /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_model.bin --load-path_adapter /content/gdrive/MyDrive/master_hpi/NLP_Project/code/models/sst/pytorch_adapter.bin --input-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/ --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json
#!python3 evaluation.py --no-cuda False --output-dir /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/ -- skip_intrasentence False --skip_intersentence False --batch_size 1

In [ ]:
!python3 /content/gdrive/MyDrive/master_hpi/NLP_Project/code/adapter_evaluation/score_evaluation.py --gold-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/dev.json --predictions-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/predictions.json --output-file /content/gdrive/MyDrive/master_hpi/NLP_Project/code/results/sst/summary.json